In [1]:
from Database import Database
import sys
# import multiprocessing as mp
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np

In [2]:
def get_project_ids(db):
    db.cursor.execute("""SELECT *
                            FROM project""")
    projects = db.cursor.fetchall()
    dict_project = dict()
    for proj in projects:
        dict_project[proj['ID']] = proj['project_ID']
    return dict_project

In [3]:
 path_db = 'D:/Hanze_Groningen/STAGE/db_laatste_copy.db' #'D:/Hanze_Groningen/STAGE/DATAB/copydatabase_C.db'
# Database connection
db = Database(path_db)
dict_project = get_project_ids(db)
print(len(dict_project))

43


In [4]:
df = pd.read_sql('''SELECT project.cancer, sum_dosage_GT.donor_project_ID, 
                            sum_dosage_GT.donor_ID, sum_dosage_GT.snp_ID, 
                            snp.chr, snp.pos_start, snp.pos_end 
                    FROM project, sum_dosage_GT, snp 
                    WHERE sum_dosage_GT.snp_ID=snp.ID AND 
                              sum_dosage_GT.donor_project_ID = project.ID AND 
                              (sum_dosage_GT.GT2 = 1 OR sum_dosage_GT.GT2 = 2) AND 
                              sum_dosage_GT.total_read_count_sum >= 33;''', db.mydb_connection)

In [5]:
print('HEAD')
print(df.head())
print(len(set(df['cancer'])))
print(set(df['cancer']))
print(len(set(df['donor_project_ID'])))
print(set(df['donor_project_ID']))
print(len(set(df['donor_ID'])))
print(len(set(df['snp_ID'])))
print(len(df))
print(set(df['chr']))

HEAD
          cancer  donor_project_ID  donor_ID  snp_ID chr  pos_start    pos_end
0  Head and neck                 1         1      41  11  101465614  101465614
1  Head and neck                 1         1     151  11  103127113  103127113
2  Head and neck                 1         1     197   8   82046917   82046917
3  Head and neck                 1         1     216  11  107789760  107789760
4  Head and neck                 1         1     241  14   48732779   48732779
15
{'Blood', 'Gall Bladder', 'Liver', 'Bladder', 'Brain', 'Ovary', 'Bone', 'Cervix', 'Uterus', 'Colorectal', 'Mesenchymal', 'Head and neck', 'Kidney', 'Stomach', 'Breast'}
40
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43}
2169
5651989
5855204
{'3', '1', '5', '19', '21', '13', '17', '15', '11', '6', '2', 'Y', 'MT', '9', 'X', '10', '20', '4', '18', '14', '7', '22', '12', '16', '8'}


In [6]:
df_fil = df.loc[df['chr'] != 'MT']
df_fil['chr'] ='chr' + df_fil['chr'].astype(str)

<ipython-input-6-5a2c1653c4d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['chr'] ='chr' + df_fil['chr'].astype(str)


In [7]:
breast_cancer = df_fil.loc[df_fil['cancer'] == 'Breast']
breast_cancer1 = breast_cancer[['chr', 'pos_start', 'pos_end']]
breast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)

<ipython-input-7-bb9243d632d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)


In [8]:
nonbreast_cancer = df_fil.loc[df_fil['cancer'] != 'Breast']
nonbreast_cancer1 = nonbreast_cancer[['chr', 'pos_start', 'pos_end']]
nonbreast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)

<ipython-input-8-4f38133a77e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbreast_cancer1.sort_values(by=['chr', 'pos_start'], inplace=True)


In [9]:
breast_cancer['snp'] = breast_cancer['chr'] + '_' + breast_cancer['pos_start'].map(str) + '_' + breast_cancer['pos_end'].map(str)
print(len(list(set(breast_cancer['donor_ID']))))
num_donor_breast = len(list(set(breast_cancer['donor_ID'])))
breast_cancer

285


<ipython-input-9-8adcad4a8fd6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breast_cancer['snp'] = breast_cancer['chr'] + '_' + breast_cancer['pos_start'].map(str) + '_' + breast_cancer['pos_end'].map(str)


,cancer,donor_project_ID,donor_ID,snp_ID,chr,pos_start,pos_end,snp
776755,Breast,25,1305,1979425,chr9,104350785,104350785,chr9_104350785_104350785
776756,Breast,25,1305,3656805,chr1,25646616,25646616,chr1_25646616_25646616
776757,Breast,25,1305,3656808,chr1,25646640,25646640,chr1_25646640_25646640
776758,Breast,25,1305,3656812,chr1,29155652,29155652,chr1_29155652_29155652
776759,Breast,25,1305,3656831,chr1,61862224,61862224,chr1_61862224_61862224
...,...,...,...,...,...,...,...,...
993973,Breast,28,2057,8740839,chr8,48761994,48761994,chr8_48761994_48761994
993974,Breast,28,2057,8740853,chr2,54494456,54494456,chr2_54494456_54494456
993975,Breast,28,2057,8740862,chr22,46712078,46712078,chr22_46712078_46712078
993976,Breast,28,2057,8740870,chr17,33772618,33772618,chr17_33772618_33772618


In [10]:
nonbreast_cancer['snp'] = nonbreast_cancer['chr'] + '_' + nonbreast_cancer['pos_start'].map(str) + '_' + nonbreast_cancer['pos_end'].map(str)
print(len(list(set(nonbreast_cancer['donor_ID']))))
num_donor_nonbreast = len(list(set(nonbreast_cancer['donor_ID'])))
nonbreast_cancer

<ipython-input-10-a350e5104874>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbreast_cancer['snp'] = nonbreast_cancer['chr'] + '_' + nonbreast_cancer['pos_start'].map(str) + '_' + nonbreast_cancer['pos_end'].map(str)


1884


,cancer,donor_project_ID,donor_ID,snp_ID,chr,pos_start,pos_end,snp
0,Head and neck,1,1,41,chr11,101465614,101465614,chr11_101465614_101465614
1,Head and neck,1,1,151,chr11,103127113,103127113,chr11_103127113_103127113
2,Head and neck,1,1,197,chr8,82046917,82046917,chr8_82046917_82046917
3,Head and neck,1,1,216,chr11,107789760,107789760,chr11_107789760_107789760
4,Head and neck,1,1,241,chr14,48732779,48732779,chr14_48732779_48732779
...,...,...,...,...,...,...,...,...
5855199,Mesenchymal,43,2996,21812208,chr7,102949431,102949431,chr7_102949431_102949431
5855200,Mesenchymal,43,2996,21812224,chr11,121028837,121028837,chr11_121028837_121028837
5855201,Mesenchymal,43,2996,21812247,chr1,155658167,155658167,chr1_155658167_155658167
5855202,Mesenchymal,43,2996,21812248,chr17,46607840,46607840,chr17_46607840_46607840


In [11]:
breast_dict = dict(Counter(list(breast_cancer['snp'])))
nonbreast_dict = dict(Counter(list(nonbreast_cancer['snp'])))


In [12]:
breast_snp_count_df = pd.DataFrame([breast_dict.keys(), breast_dict.values()]).T
breast_snp_count_df.columns= ['snp', 'counts']

max_breast = breast_snp_count_df['counts'].max()
breast_snp_count_df['counts'].max()

22

In [13]:
breast_snp_count_df

,snp,counts
0,chr9_104350785_104350785,1
1,chr1_25646616_25646616,1
2,chr1_25646640_25646640,1
3,chr1_29155652_29155652,1
4,chr1_61862224_61862224,1
...,...,...
217123,chr8_48761994_48761994,1
217124,chr2_54494456_54494456,1
217125,chr22_46712078_46712078,1
217126,chr17_33772618_33772618,1


In [14]:
nonbreast_snp_count_df = pd.DataFrame([nonbreast_dict.keys(), nonbreast_dict.values()]).T
nonbreast_snp_count_df.columns= ['snp', 'counts']

max_nonbreast = nonbreast_snp_count_df['counts'].max()
nonbreast_snp_count_df['counts'].max()

39

In [ ]:
nonbreast_snp_count_df

In [ ]:
breast_snp_count_df[['chr', 'pos_start', 'pos_end']] = breast_snp_count_df['snp'].str.split('_', expand=True)
nonbreast_snp_count_df[['chr', 'pos_start', 'pos_end']] = nonbreast_snp_count_df['snp'].str.split('_', expand=True)

In [ ]:
breast_snp_count_df

In [ ]:
nonbreast_snp_count_df

In [ ]:
breast_snp_count_df.drop('snp', axis=1, inplace=True)
nonbreast_snp_count_df.drop('snp', axis=1, inplace=True)

In [ ]:
sort_breast_snp_count_df = breast_snp_count_df.sort_values('counts', ascending=False)
sort_nonbreast_snp_count_df = nonbreast_snp_count_df.sort_values('counts', ascending=False)

In [ ]:
sort_breast_snp_count_df.rename(columns={'counts': 'counts_breast'}, inplace=True)
sort_nonbreast_snp_count_df.rename(columns={'counts': 'counts_nonbreast'}, inplace=True)

In [ ]:
sort_breast_snp_count_df

In [ ]:
sort_nonbreast_snp_count_df

In [ ]:
sort_breast_snp_count_df.to_csv(f"D:/Hanze_Groningen/STAGE/R/breast_snp_count_df_maxcount_{max_breast}_donor_{num_donor_breast}_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)
sort_nonbreast_snp_count_df.to_csv(f"D:/Hanze_Groningen/STAGE/R/nonbreast_snp_count_df_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
# merge_b_nb = pd.concat([sort_breast_snp_count_df, sort_nonbreast_snp_count_df], ignore_index=True, sort=False)

#https://stackoverflow.com/questions/53645882/pandas-merging-101
merge_b_nb = sort_breast_snp_count_df.merge(sort_nonbreast_snp_count_df, on=['chr', 'pos_start', 'pos_end'], how='outer')

In [ ]:
merge_b_nb[(merge_b_nb['counts_breast']>=1) & (merge_b_nb['counts_nonbreast']>=1)]

In [ ]:
merge_b_nb

In [ ]:
merge_b_nb.to_csv(f"D:/Hanze_Groningen/STAGE/R/ALL_nan_df_BREAST_maxcount_{max_breast}_donor_{num_donor_breast}_NONBREAST_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_BOTH_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
merge_b_nb_0 = merge_b_nb.fillna(0)
merge_b_nb_0.to_csv(f"D:/Hanze_Groningen/STAGE/R/ALL_0_df_BREAST_maxcount_{max_breast}_donor_{num_donor_breast}_NONBREAST_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_BOTH_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
ax = sns.boxplot(data=sort_breast_snp_count_df[['counts_breast']]).set(ylabel='#snps')
plt.show()
plt.clf()
ax = sns.displot(sort_breast_snp_count_df[['counts_breast']])
plt.show()
plt.clf()

In [ ]:
filter_breast = sort_breast_snp_count_df[(sort_breast_snp_count_df['counts_breast']>1)]

ax = sns.boxplot(data=filter_breast[['counts_breast']]).set(ylabel='#snps')
plt.show()
plt.clf()
ax = sns.displot(filter_breast[['counts_breast']])
plt.show()
plt.clf()

In [ ]:
ax = sns.boxplot(data=sort_nonbreast_snp_count_df[['counts_nonbreast']]).set(ylabel='#snps')
plt.show()
plt.clf()
ax = sns.displot(sort_nonbreast_snp_count_df[['counts_nonbreast']])
plt.show()
plt.clf()

In [ ]:
filter_nonbreast = sort_nonbreast_snp_count_df[(sort_nonbreast_snp_count_df['counts_nonbreast']>1)]

ax = sns.boxplot(data=filter_nonbreast[['counts_nonbreast']]).set(ylabel='#snps')
plt.show()
plt.clf()
ax = sns.displot(filter_nonbreast[['counts_nonbreast']])
plt.show()
plt.clf()

In [ ]:
merge_b_nb_0

In [ ]:
# https://www.geeksforgeeks.org/python-pearsons-chi-square-test/
from scipy.stats import chi2_contingency
  
# defining the table
#
#       donor   |  snp
#  b  | 285     | 22
#  nb | 1885    | 7
data = [[285, 22], [1885, 7]]
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

In [ ]:
f = open("D:/Hanze_Groningen/STAGE/R/chi_square.tsv", 'w')
for index, row in merge_b_nb_0.iterrows():
    data = [[285, row['counts_breast']], [1885, row['counts_nonbreast']]]
    stat, p, dof, expected = chi2_contingency(data)
    f.write(f"{row['chr']}\t{row['pos_start']}\t{row['pos_end']}\t{row['counts_breast']}\t{row['counts_nonbreast']}\t{stat}\t{p}\t{dof}\t{expected}\n")
    
f.close()

In [ ]:
fold_change = merge_b_nb_0.copy()
fold_change['filter_b'] = fold_change['counts_breast'] / 285
fold_change['filter_nb'] = fold_change['counts_nonbreast'] / 1885
fold_change['FC'] = fold_change['filter_b'] / folc_change['filter_nb']

In [ ]:
#tsjerk

# X^2
# f = n / n.sum()
# E1 = S.sum(axis=1)[:,None]*f
# C = (n-s)
# E2 = C.sum(axis=1)[:,None]*f
# D1 = ((E1-S)**2/E1).sum(axis=1)
# D2 = ((E2-C)/E2).sum(axis=1)
# X2 = D1 + D2

# log_fc = np.diff(np.log(S), axis = 1) - (log n).sum()

In [ ]:
n = np.array([285, 1885])
S = merge_b_nb_0[['counts_breast', 'counts_nonbreast']].to_numpy()
S

In [ ]:
f = n / n.sum()
f

In [ ]:
E1 = S.sum(axis=1)[:,None]*f
E1

In [ ]:
C = (n-S)
C

In [ ]:
E2 = C.sum(axis=1)[:,None]*f
E2

In [ ]:
D1 = ((E1-S)**2/E1).sum(axis=1)
D1

In [ ]:
D2 = ((E2-C)/E2).sum(axis=1)
D2

In [ ]:
X2 = D1 + D2
X2

In [ ]:
# chi2.sf(100.07565261,1)
# chi2.sf(0.1508023,1)

In [ ]:
# https://stackoverflow.com/questions/11725115/p-value-from-chi-sq-test-statistic-in-python
from scipy.stats.distributions import chi2
p_value_X2 = chi2.sf(X2,1)

In [ ]:
# np.log(285)+np.log(1885) == (np.log(n)).sum()
# np.log(22)-np.log(7)-np.log(285)-np.log(1885)

In [ ]:
# logS = np.log2(S)
# logS

# log_fc = np.diff(np.log2(S), axis = 1) - (np.log2(n)).sum()

In [ ]:
# np.log(7)-np.log(22) = np.diff(logS)

In [ ]:
log2_fc = np.diff(np.log(S), axis = 1) - (np.log(n)).sum()
log2_fc

In [ ]:
log10_p_value = -np.log10(p_value_X2)
log10_p_value

In [ ]:
merge_b_nb_0['X2'] = X2
merge_b_nb_0['p_value_X2'] = p_value_X2
merge_b_nb_0['log10_p_value'] = log10_p_value # -log10(p-value)
merge_b_nb_0['log2_fc'] = log2_fc # log2(FC)

In [ ]:
merge_b_nb_0

In [ ]:
merge_b_nb_0.to_csv(f"D:/Hanze_Groningen/STAGE/R/p_valueANDfc_df_BREAST_maxcount_{max_breast}_donor_{num_donor_breast}_NONBREAST_maxcount_{max_nonbreast}_donor_{num_donor_nonbreast}_BOTH_GT12_totalread_33.tsv", sep='\t', encoding='utf-8', index=False)

In [ ]:
# https://github.com/vappiah/bioinfoscripts/blob/main/volcano-plot-in-python.py
from bioinfokit import analys, visuz
#plot and save. Plot will be saved in the current working directory
visuz.gene_exp.volcano(df=merge_b_nb_0,lfc='log2_fc',pv='p_value_X2')

#plot and open . This will not save the image
visuz.gene_exp.volcano(df=merge_b_nb_0,lfc='log2_fc',pv='p_value_X2',show=True)

#plot and rotate x-axis labels
visuz.gene_exp.volcano(df=merge_b_nb_0,lfc='log2_fc',pv='p_value_X2',ar=0,show=True)

#show statistically significant differentially expressed genes based on thresholds
visuz.gene_exp.volcano(df=merge_b_nb_0,lfc='log2_fc',pv='p_value_X2',ar=0,show=True,plotlegend=True,
                        legendpos='upper right',lfc_thr=(1,2),pv_thr=(0.05,0.01))

#change plot color
visuz.gene_exp.volcano(df=merge_b_nb_0,lfc='log2_fc',pv='p_value_X2',ar=0,color=('blue','grey','green'),show=True)


#Add gene labels
# visuz.gene_exp.volcano(df=merge_b_nb_0,lfc="log2_fc",pv="p_value_X2",geneid="GeneNames"
#                         ,genenames=("LOC_Os09g01000.1", "LOC_Os01g50030.1", 
#                                     "LOC_Os06g40940.3"),ar=0,show=True)
